# Módulo 3: Context Engineering
## Uso de Herramientas y Fuentes Externas para Mejorar Capacidades de IA

**Duración estimada**: 1 hora  
**Nivel**: Avanzado  
**Objetivo**: Implementar context engineering con herramientas externas y RAG

---

## 📚 Contenido del Módulo

Este notebook cubre:
1. **Práctica 3.1**: RAG básico con documentos
2. **Práctica 3.2**: RAG avanzado con múltiples fuentes
3. **Práctica 3.3**: Sistema completo de context engineering

---

## 🎯 Objetivos de Aprendizaje

Al finalizar este módulo serás capaz de:
- Implementar sistemas RAG (Retrieval-Augmented Generation) básicos
- Usar embeddings y búsqueda semántica para encontrar información relevante
- Integrar múltiples fuentes de contexto (documentos, bases de datos, APIs)
- Construir sistemas completos de context engineering para producción
- Optimizar sistemas RAG para mejor rendimiento

---

## ⚙️ Configuración Inicial

Antes de comenzar, asegúrate de tener:
- Python 3.8+ instalado
- Cuenta de OpenAI con API key
- Librerías necesarias instaladas

**Instalación en Windows:**
```bash
pip install openai python-dotenv sentence-transformers chromadb langchain
```

**Nota**: La instalación de `sentence-transformers` puede tardar varios minutos la primera vez.

In [ ]:
# Instalación de dependencias (ejecutar solo si es necesario)
# !pip install openai python-dotenv sentence-transformers chromadb langchain

# Importar librerías necesarias
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Any, Tuple
import hashlib

# Cargar variables de entorno
load_dotenv()

# Inicializar cliente de OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Verificar configuración
if not client.api_key:
    print("⚠️ ADVERTENCIA: No se encontró OPENAI_API_KEY. Configúrala en un archivo .env")
else:
    print("✅ Cliente de OpenAI configurado correctamente")

# Intentar importar librerías opcionales
try:
    from sentence_transformers import SentenceTransformer
    import chromadb
    from chromadb.config import Settings
    EMBEDDINGS_AVAILABLE = True
    print("✅ Librerías de embeddings disponibles")
except ImportError:
    EMBEDDINGS_AVAILABLE = False
    print("⚠️ Algunas librerías no están disponibles. Algunas funciones pueden no funcionar.")
    print("   Instala con: pip install sentence-transformers chromadb")

---

# Práctica 3.1: RAG Básico con Documentos

## 🎯 Objetivo
Implementar un sistema RAG (Retrieval-Augmented Generation) simple que permita hacer preguntas sobre documentos usando embeddings y búsqueda semántica.

## 📖 Contexto y Teoría

### ¿Qué es RAG?

**RAG (Retrieval-Augmented Generation)** es una técnica que:
1. **Retrieval (Recuperación)**: Busca información relevante en una base de conocimiento
2. **Augmentation (Aumento)**: Añade esa información al prompt
3. **Generation (Generación)**: El modelo genera respuestas usando el contexto recuperado

### ¿Por qué usar RAG?

- ✅ **Información actualizada**: Acceso a datos que el modelo no tiene en su entrenamiento
- ✅ **Fuentes verificables**: Puedes citar de dónde viene la información
- ✅ **Menor alucinación**: El modelo tiene contexto real para responder
- ✅ **Especialización**: Puedes usar conocimiento específico del dominio

### Arquitectura RAG Básica

```
1. Documentos → Chunking (dividir en fragmentos)
2. Chunks → Embeddings (convertir a vectores)
3. Almacenar en Vector Store
4. Query → Embedding de la pregunta
5. Búsqueda semántica → Encontrar chunks relevantes
6. Chunks + Pregunta → Prompt al modelo
7. Respuesta generada
```

### Casos de Uso
- Q&A sobre documentación técnica
- Asistentes con conocimiento corporativo
- Búsqueda en código legacy
- Sistemas de ayuda contextual

## 🔍 Ejercicio Guiado: Sistema RAG Básico

Vamos a crear un sistema RAG simple paso a paso.

In [ ]:
# EJEMPLO: Sistema RAG básico usando OpenAI Embeddings

# Documentos de ejemplo (en producción vendrían de archivos, bases de datos, etc.)
documentos = [
    "Python es un lenguaje de programación de alto nivel, interpretado y de propósito general. Fue creado por Guido van Rossum y se lanzó por primera vez en 1991.",
    "OpenAI es una empresa de investigación en inteligencia artificial fundada en 2015. Es conocida por desarrollar modelos como GPT-3, GPT-4 y DALL-E.",
    "Los embeddings son representaciones vectoriales de texto que capturan el significado semántico. Permiten comparar textos por similitud en lugar de coincidencia exacta.",
    "RAG (Retrieval-Augmented Generation) combina búsqueda de información con generación de texto. Primero busca información relevante, luego la usa para generar respuestas.",
    "Jupyter Notebook es un entorno interactivo que permite combinar código, texto y visualizaciones. Es muy popular en ciencia de datos y análisis exploratorio."
]

def crear_embeddings(textos: List[str]) -> List[List[float]]:
    """
    Crea embeddings usando OpenAI
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",  # Modelo económico y eficiente
        input=textos
    )
    return [item.embedding for item in response.data]

def calcular_similitud_coseno(vec1: List[float], vec2: List[float]) -> float:
    """
    Calcula similitud coseno entre dos vectores
    """
    import math
    
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(a * a for a in vec1))
    magnitude2 = math.sqrt(sum(a * a for a in vec2))
    
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    return dot_product / (magnitude1 * magnitude2)

def buscar_documentos_relevantes(pregunta: str, documentos: List[str], top_k: int = 2) -> List[Tuple[str, float]]:
    """
    Busca los documentos más relevantes para una pregunta
    """
    # Crear embedding de la pregunta
    embedding_pregunta = crear_embeddings([pregunta])[0]
    
    # Crear embeddings de todos los documentos
    embeddings_docs = crear_embeddings(documentos)
    
    # Calcular similitud con cada documento
    similitudes = []
    for doc, emb in zip(documentos, embeddings_docs):
        similitud = calcular_similitud_coseno(embedding_pregunta, emb)
        similitudes.append((doc, similitud))
    
    # Ordenar por similitud y retornar top_k
    similitudes.sort(key=lambda x: x[1], reverse=True)
    return similitudes[:top_k]

def sistema_rag_basico(pregunta: str, documentos: List[str]) -> str:
    """
    Sistema RAG básico completo
    """
    print(f"❓ Pregunta: {pregunta}\n")
    
    # Paso 1: Buscar documentos relevantes
    print("🔍 Buscando documentos relevantes...")
    documentos_relevantes = buscar_documentos_relevantes(pregunta, documentos, top_k=2)
    
    print(f"✅ Encontrados {len(documentos_relevantes)} documentos relevantes:")
    for i, (doc, sim) in enumerate(documentos_relevantes, 1):
        print(f"   {i}. Similitud: {sim:.3f} - {doc[:60]}...")
    
    # Paso 2: Construir contexto
    contexto = "\n\n".join([doc for doc, _ in documentos_relevantes])
    
    # Paso 3: Generar respuesta usando el contexto
    print("\n🤖 Generando respuesta...")
    prompt = f"""
Basándote en la siguiente información, responde la pregunta.
Si la información no es suficiente, di que no tienes suficiente información.

Información:
{contexto}

Pregunta: {pregunta}

Responde de manera clara y concisa:
"""
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    
    respuesta = response.choices[0].message.content
    return respuesta

# Probar el sistema RAG
print("=" * 60)
print("SISTEMA RAG BÁSICO")
print("=" * 60 + "\n")

preguntas = [
    "¿Qué es Python?",
    "¿Qué es RAG y cómo funciona?",
    "¿Quién creó Python y cuándo?"
]

for pregunta in preguntas:
    respuesta = sistema_rag_basico(pregunta, documentos)
    print(f"\n💬 Respuesta: {respuesta}\n")
    print("-" * 60 + "\n")

In [ ]:
# ESPACIO PARA TU CÓDIGO
def chunk_documento(documento: str, tamano_chunk: int = 200) -> List[Dict[str, Any]]:
    """
    Divide un documento en chunks de tamaño aproximado
    Retorna lista de chunks con metadatos
    """
    # TODO: Implementa el chunking
    # - Divide el documento en palabras
    # - Agrupa en chunks de ~tamano_chunk palabras
    # - Incluye metadatos (índice, documento original)
    pass

def sistema_rag_mejorado(pregunta: str, documentos: List[str]) -> str:
    """
    Sistema RAG mejorado con chunking
    """
    # TODO: Implementa el sistema mejorado
    # 1. Chunkear todos los documentos
    # 2. Crear embeddings de los chunks
    # 3. Buscar chunks relevantes
    # 4. Generar respuesta con referencias
    pass

# Documentos de prueba (más largos)
documentos_largos = [
    """
    Python es un lenguaje de programación de alto nivel, interpretado y de propósito general. 
    Fue creado por Guido van Rossum y se lanzó por primera vez en 1991. Python tiene una filosofía 
    de diseño que enfatiza la legibilidad del código, notablemente mediante el uso de espacios en 
    blanco significativos. Sus construcciones de lenguaje y su enfoque orientado a objetos tienen 
    como objetivo ayudar a los programadores a escribir código claro y lógico para proyectos 
    pequeños y grandes. Python es dinámicamente tipado y recolecta basura. Soporta múltiples 
    paradigmas de programación, incluyendo programación estructurada, orientada a objetos y funcional.
    """,
    # Agrega más documentos largos aquí
]

# Prueba el sistema
# respuesta = sistema_rag_mejorado("¿Cuáles son las características principales de Python?", documentos_largos)
# print(respuesta)

## 🔥 Ejercicio de Desafío 3.1 (Opcional)

Crea un sistema RAG que:
- Use ChromaDB o FAISS para almacenamiento vectorial persistente
- Implemente re-ranking de resultados (usar múltiples estrategias de búsqueda)
- Incluya filtrado por metadatos (fecha, categoría, etc.)
- Genere respuestas con citas numeradas a los documentos fuente

**Desafío extra**: Implementa un sistema de actualización incremental (agregar nuevos documentos sin re-indexar todo).

---

# Práctica 3.2: RAG Avanzado con Múltiples Fuentes

## 🎯 Objetivo
Integrar múltiples fuentes de contexto (documentos, bases de datos, APIs) en un sistema RAG avanzado.

## 📖 Contexto y Teoría

### ¿Por qué Múltiples Fuentes?

En sistemas reales, la información puede venir de:
- **Documentos**: PDFs, Markdown, texto plano
- **Bases de datos**: SQL, NoSQL, datos estructurados
- **APIs**: Servicios externos, datos en tiempo real
- **Código**: Repositorios, documentación técnica

### Arquitectura Multi-Fuente

```
Fuente 1 (Docs) → Embeddings → Vector Store
Fuente 2 (DB)   → Query SQL   → Resultados
Fuente 3 (API)  → HTTP Request → Datos JSON
                    ↓
            Agregador de Contexto
                    ↓
            Prompt con Todo el Contexto
                    ↓
            Respuesta Final
```

### Estrategias de Agregación

1. **Fusión temprana**: Combinar antes de buscar
2. **Fusión tardía**: Buscar en cada fuente, luego combinar resultados
3. **Híbrida**: Combinar búsqueda semántica + búsqueda estructurada

### Casos de Uso
- Sistemas de consulta corporativa
- Asistentes con acceso a múltiples sistemas
- Análisis de datos multi-fuente
- Sistemas de recomendación

## 🔍 Ejercicio Guiado: Sistema RAG Multi-Fuente

Vamos a crear un sistema que combine documentos, base de datos simulada y API simulada.

In [ ]:
# EJEMPLO: Sistema RAG con múltiples fuentes

# Fuente 1: Documentos (ya implementado)
documentos = [
    "Python es un lenguaje de programación versátil usado en ciencia de datos, web development y automatización.",
    "OpenAI proporciona APIs para integración de IA en aplicaciones mediante modelos como GPT-4.",
    "Los embeddings permiten búsqueda semántica encontrando textos similares en significado."
]

# Fuente 2: Base de datos simulada
base_datos_usuarios = {
    "usuarios": [
        {"id": 1, "nombre": "Juan", "rol": "desarrollador", "proyecto": "API Backend"},
        {"id": 2, "nombre": "María", "rol": "data scientist", "proyecto": "ML Models"},
        {"id": 3, "nombre": "Carlos", "rol": "devops", "proyecto": "Infrastructure"}
    ],
    "proyectos": [
        {"id": 1, "nombre": "API Backend", "tecnologia": "Python, FastAPI", "estado": "activo"},
        {"id": 2, "nombre": "ML Models", "tecnologia": "Python, TensorFlow", "estado": "activo"},
        {"id": 3, "nombre": "Infrastructure", "tecnologia": "Docker, Kubernetes", "estado": "planning"}
    ]
}

def consultar_base_datos(query: str) -> str:
    """
    Simula consulta a base de datos basada en palabras clave
    """
    query_lower = query.lower()
    resultados = []
    
    # Búsqueda simple por palabras clave
    if "usuario" in query_lower or "persona" in query_lower:
        for usuario in base_datos_usuarios["usuarios"]:
            if any(palabra in str(usuario).lower() for palabra in query_lower.split()):
                resultados.append(f"Usuario: {usuario['nombre']}, Rol: {usuario['rol']}, Proyecto: {usuario['proyecto']}")
    
    if "proyecto" in query_lower:
        for proyecto in base_datos_usuarios["proyectos"]:
            if any(palabra in str(proyecto).lower() for palabra in query_lower.split()):
                resultados.append(f"Proyecto: {proyecto['nombre']}, Tecnología: {proyecto['tecnologia']}, Estado: {proyecto['estado']}")
    
    return "\n".join(resultados) if resultados else "No se encontraron resultados en la base de datos."

# Fuente 3: API simulada
def consultar_api(tema: str) -> str:
    """
    Simula consulta a API externa
    """
    # Simulación de datos de API
    datos_api = {
        "python": "Python 3.11 es la versión más reciente con mejoras en rendimiento y nuevas características.",
        "openai": "OpenAI API está disponible 24/7 con rate limits según el plan de suscripción.",
        "rag": "RAG es una técnica que combina recuperación de información con generación de texto."
    }
    
    for clave, valor in datos_api.items():
        if clave in tema.lower():
            return f"Datos de API: {valor}"
    
    return "No hay información disponible en la API para este tema."

def sistema_rag_multifuente(pregunta: str) -> str:
    """
    Sistema RAG que combina múltiples fuentes
    """
    print(f"❓ Pregunta: {pregunta}\n")
    
    contexto_completo = []
    fuentes_usadas = []
    
    # Fuente 1: Búsqueda en documentos (RAG semántico)
    print("🔍 Buscando en documentos...")
    documentos_relevantes = buscar_documentos_relevantes(pregunta, documentos, top_k=2)
    if documentos_relevantes:
        contexto_docs = "\n".join([doc for doc, _ in documentos_relevantes])
        contexto_completo.append(f"DOCUMENTOS:\n{contexto_docs}")
        fuentes_usadas.append("documentos")
        print(f"✅ Encontrados {len(documentos_relevantes)} documentos relevantes")
    
    # Fuente 2: Consulta a base de datos
    print("\n💾 Consultando base de datos...")
    resultado_db = consultar_base_datos(pregunta)
    if resultado_db and "No se encontraron" not in resultado_db:
        contexto_completo.append(f"BASE DE DATOS:\n{resultado_db}")
        fuentes_usadas.append("base de datos")
        print("✅ Datos encontrados en base de datos")
    
    # Fuente 3: Consulta a API
    print("\n🌐 Consultando API externa...")
    resultado_api = consultar_api(pregunta)
    if resultado_api and "No hay información" not in resultado_api:
        contexto_completo.append(f"API EXTERNA:\n{resultado_api}")
        fuentes_usadas.append("API")
        print("✅ Datos obtenidos de API")
    
    # Combinar todo el contexto
    contexto_final = "\n\n".join(contexto_completo)
    
    if not contexto_final:
        return "No se encontró información relevante en ninguna fuente disponible."
    
    # Generar respuesta
    print(f"\n🤖 Generando respuesta usando {len(fuentes_usadas)} fuente(s)...")
    prompt = f"""
Basándote en la siguiente información de múltiples fuentes, responde la pregunta de manera completa.

Información disponible:
{contexto_final}

Pregunta: {pregunta}

Responde de manera clara, citando de qué fuente viene cada parte de la información cuando sea relevante.
"""
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    
    return response.choices[0].message.content

# Probar el sistema multi-fuente
print("=" * 60)
print("SISTEMA RAG MULTI-FUENTE")
print("=" * 60 + "\n")

preguntas = [
    "¿Qué es Python y quién lo usa en la empresa?",
    "¿Qué proyectos están activos y qué tecnologías usan?",
    "¿Cómo funciona RAG según la documentación y la API?"
]

for pregunta in preguntas:
    respuesta = sistema_rag_multifuente(pregunta)
    print(f"\n💬 Respuesta:\n{respuesta}\n")
    print("=" * 60 + "\n")

In [ ]:
# ESPACIO PARA TU CÓDIGO

# Código fuente simulado
codigo_fuente = {
    "calculadora.py": """
def suma(a, b):
    return a + b

def multiplicar(a, b):
    return a * b
""",
    "utils.py": """
def procesar_datos(datos):
    resultados = []
    for item in datos:
        resultados.append(item * 2)
    return resultados
"""
}

# TODO: Implementa las funciones de búsqueda
def buscar_en_codigo(pregunta: str) -> str:
    """Busca funciones/clases relevantes en el código"""
    # Tu implementación aquí
    pass

def calcular_metricas_codigo(archivo: str, codigo: str) -> Dict[str, Any]:
    """Calcula métricas de código"""
    # Tu implementación aquí
    pass

def sistema_rag_codigo_docs(pregunta: str) -> str:
    """
    Sistema RAG que integra código, documentación y métricas
    """
    # TODO: Implementa el sistema completo
    # 1. Analizar la pregunta para determinar qué fuentes usar
    # 2. Buscar en cada fuente relevante
    # 3. Agregar resultados
    # 4. Generar respuesta consolidada
    pass

# Prueba el sistema
# preguntas_test = [
#     "¿Cómo funciona la función suma?",
#     "¿Cuál es la complejidad del código en calculadora.py?",
#     "¿Qué funciones hay disponibles para procesar datos?"
# ]
# 
# for pregunta in preguntas_test:
#     print(sistema_rag_codigo_docs(pregunta))

## 🔥 Ejercicio de Desafío 3.2 (Opcional)

Crea un sistema RAG que:
- Use un router inteligente que decida qué fuente consultar primero
- Implemente búsqueda híbrida (semántica + keyword)
- Incluya sistema de scoring para rankear resultados de múltiples fuentes
- Genere respuestas con confianza score basado en calidad de las fuentes

**Desafío extra**: Implementa un sistema de feedback que aprenda qué fuentes son más útiles para cada tipo de pregunta.

---

# Práctica 3.3: Sistema Completo de Context Engineering

## 🎯 Objetivo
Construir una solución end-to-end de context engineering con RAG, validación, optimización y características de producción.

## 📖 Contexto y Teoría

### Componentes de un Sistema Completo

1. **Ingestión de datos**: Cargar y procesar múltiples tipos de documentos
2. **Chunking inteligente**: Dividir documentos preservando contexto
3. **Embeddings y almacenamiento**: Vector store persistente
4. **Búsqueda avanzada**: Múltiples estrategias de retrieval
5. **Generación con contexto**: Prompt engineering optimizado
6. **Validación y calidad**: Verificar respuestas
7. **Monitoreo**: Tracking de uso y performance

### Optimizaciones Importantes

- **Chunking estratégico**: Tamaño óptimo según tipo de documento
- **Re-ranking**: Mejorar resultados de búsqueda inicial
- **Caché**: Evitar recálculos costosos
- **Filtrado**: Usar metadatos para refinar búsquedas
- **Hybrid search**: Combinar búsqueda semántica y keyword

### Casos de Uso
- Asistentes corporativos con conocimiento interno
- Sistemas de Q&A sobre documentación técnica
- Agentes con memoria persistente
- Sistemas de recomendación basados en contenido

## 🔍 Ejercicio Guiado: Sistema Completo de Context Engineering

Vamos a construir un sistema completo con todas las características.

In [ ]:
# EJEMPLO: Sistema completo de context engineering

class SistemaContextEngineering:
    """
    Sistema completo de context engineering con todas las características
    """
    
    def __init__(self):
        self.documentos = []
        self.embeddings_cache = {}  # Caché de embeddings
        self.queries_cache = {}     # Caché de queries
        
    def agregar_documento(self, documento: str, metadatos: Dict = None):
        """Agrega un documento al sistema"""
        doc_id = hashlib.md5(documento.encode()).hexdigest()[:8]
        self.documentos.append({
            "id": doc_id,
            "texto": documento,
            "metadatos": metadatos or {}
        })
        return doc_id
    
    def chunk_inteligente(self, texto: str, tamano: int = 200, overlap: int = 50) -> List[Dict]:
        """
        Chunking inteligente con overlap para preservar contexto
        """
        palabras = texto.split()
        chunks = []
        
        for i in range(0, len(palabras), tamano - overlap):
            chunk_palabras = palabras[i:i + tamano]
            chunk_texto = " ".join(chunk_palabras)
            
            chunks.append({
                "texto": chunk_texto,
                "inicio": i,
                "fin": min(i + tamano, len(palabras))
            })
        
        return chunks
    
    def obtener_embedding(self, texto: str, usar_cache: bool = True) -> List[float]:
        """Obtiene embedding con caché"""
        if usar_cache:
            texto_hash = hashlib.md5(texto.encode()).hexdigest()
            if texto_hash in self.embeddings_cache:
                return self.embeddings_cache[texto_hash]
        
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=[texto]
        )
        embedding = response.data[0].embedding
        
        if usar_cache:
            self.embeddings_cache[texto_hash] = embedding
        
        return embedding
    
    def buscar_relevantes(self, pregunta: str, top_k: int = 3, usar_reranking: bool = True) -> List[Dict]:
        """
        Búsqueda avanzada con re-ranking opcional
        """
        # Búsqueda inicial
        embedding_pregunta = self.obtener_embedding(pregunta)
        resultados = []
        
        for doc in self.documentos:
            # Chunkear si es necesario
            if len(doc["texto"].split()) > 300:
                chunks = self.chunk_inteligente(doc["texto"])
                for chunk in chunks:
                    embedding_chunk = self.obtener_embedding(chunk["texto"])
                    similitud = calcular_similitud_coseno(embedding_pregunta, embedding_chunk)
                    resultados.append({
                        "texto": chunk["texto"],
                        "similitud": similitud,
                        "metadatos": doc["metadatos"],
                        "doc_id": doc["id"]
                    })
            else:
                embedding_doc = self.obtener_embedding(doc["texto"])
                similitud = calcular_similitud_coseno(embedding_pregunta, embedding_doc)
                resultados.append({
                    "texto": doc["texto"],
                    "similitud": similitud,
                    "metadatos": doc["metadatos"],
                    "doc_id": doc["id"]
                })
        
        # Ordenar por similitud
        resultados.sort(key=lambda x: x["similitud"], reverse=True)
        
        # Re-ranking simple: boost si hay palabras clave en común
        if usar_reranking:
            palabras_pregunta = set(pregunta.lower().split())
            for resultado in resultados[:top_k * 2]:  # Considerar más para re-ranking
                palabras_texto = set(resultado["texto"].lower().split())
                palabras_comunes = len(palabras_pregunta & palabras_texto)
                resultado["similitud"] += palabras_comunes * 0.05  # Boost pequeño
        
        # Re-ordenar después de re-ranking
        if usar_reranking:
            resultados.sort(key=lambda x: x["similitud"], reverse=True)
        
        return resultados[:top_k]
    
    def generar_respuesta(self, pregunta: str, contexto: List[Dict], validar: bool = True) -> Dict[str, Any]:
        """
        Genera respuesta con validación
        """
        # Construir contexto formateado
        contexto_texto = "\n\n".join([
            f"[Fuente {i+1}] {r['texto']}" 
            for i, r in enumerate(contexto)
        ])
        
        prompt = f"""
Eres un asistente experto que responde preguntas basándote en el contexto proporcionado.

Contexto:
{contexto_texto}

Pregunta: {pregunta}

Instrucciones:
- Responde de manera clara y precisa
- Si la información no está en el contexto, di claramente que no tienes esa información
- Cita las fuentes cuando sea relevante
- Sé conciso pero completo

Respuesta:
"""
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        
        respuesta = response.choices[0].message.content
        
        resultado = {
            "respuesta": respuesta,
            "fuentes": [r["doc_id"] for r in contexto],
            "similitudes": [r["similitud"] for r in contexto]
        }
        
        # Validación simple: verificar que la respuesta no es demasiado genérica
        if validar:
            palabras_respuesta = set(respuesta.lower().split())
            palabras_pregunta = set(pregunta.lower().split())
            overlap = len(palabras_respuesta & palabras_pregunta)
            
            if overlap < 2 and len(respuesta) < 50:
                resultado["advertencia"] = "La respuesta puede ser demasiado genérica"
        
        return resultado
    
    def consultar(self, pregunta: str, top_k: int = 3) -> Dict[str, Any]:
        """
        Método principal para consultar el sistema
        """
        print(f"❓ Pregunta: {pregunta}\n")
        
        # Buscar documentos relevantes
        print("🔍 Buscando información relevante...")
        contexto = self.buscar_relevantes(pregunta, top_k=top_k)
        
        if not contexto:
            return {
                "respuesta": "No se encontró información relevante para responder esta pregunta.",
                "fuentes": [],
                "similitudes": []
            }
        
        print(f"✅ Encontrados {len(contexto)} fragmentos relevantes")
        for i, r in enumerate(contexto, 1):
            print(f"   {i}. Similitud: {r['similitud']:.3f} - {r['texto'][:60]}...")
        
        # Generar respuesta
        print("\n🤖 Generando respuesta...")
        resultado = self.generar_respuesta(pregunta, contexto)
        
        return resultado

# Crear y probar el sistema
print("=" * 60)
print("SISTEMA COMPLETO DE CONTEXT ENGINEERING")
print("=" * 60 + "\n")

sistema = SistemaContextEngineering()

# Agregar documentos
sistema.agregar_documento(
    "Python es un lenguaje de programación de alto nivel muy popular en ciencia de datos y desarrollo web.",
    {"categoria": "programacion", "fecha": "2024"}
)
sistema.agregar_documento(
    "OpenAI desarrolla modelos de lenguaje como GPT-4 que pueden generar texto, código y responder preguntas.",
    {"categoria": "ia", "fecha": "2024"}
)
sistema.agregar_documento(
    "RAG combina recuperación de información con generación de texto para crear sistemas de IA más precisos y actualizados.",
    {"categoria": "ia", "fecha": "2024"}
)

# Consultas de prueba
preguntas = [
    "¿Qué es Python y para qué se usa?",
    "¿Cómo funciona RAG?",
    "¿Qué modelos desarrolla OpenAI?"
]

for pregunta in preguntas:
    resultado = sistema.consultar(pregunta)
    print(f"\n💬 Respuesta:\n{resultado['respuesta']}\n")
    print(f"📚 Fuentes usadas: {resultado['fuentes']}")
    if 'advertencia' in resultado:
        print(f"⚠️ {resultado['advertencia']}")
    print("=" * 60 + "\n")

In [ ]:
# ESPACIO PARA TU CÓDIGO
class SistemaContextEngineeringAvanzado(SistemaContextEngineering):
    """
    Extensión del sistema con características de producción
    """
    
    def __init__(self):
        super().__init__()
        # TODO: Agrega atributos para métricas, feedback, logs
        self.metricas = {
            "total_consultas": 0,
            "tiempo_promedio": 0,
            "consultas_exitosas": 0
        }
        self.feedback = []
        self.logs = []
    
    def consultar_con_filtros(self, pregunta: str, filtros: Dict = None, top_k: int = 3) -> Dict[str, Any]:
        """
        Consulta con filtros de metadatos
        """
        # TODO: Implementa filtrado por metadatos
        # - Filtrar documentos según metadatos antes de buscar
        # - Aplicar filtros en la búsqueda
        pass
    
    def registrar_feedback(self, consulta_id: str, util: bool, comentario: str = ""):
        """
        Registra feedback del usuario
        """
        # TODO: Implementa sistema de feedback
        pass
    
    def obtener_metricas(self) -> Dict[str, Any]:
        """
        Retorna métricas del sistema
        """
        # TODO: Calcula y retorna métricas
        pass
    
    def exportar_logs(self, archivo: str = "logs_consultas.json"):
        """
        Exporta logs a archivo JSON
        """
        # TODO: Implementa exportación de logs
        pass

# Prueba el sistema avanzado
# sistema_avanzado = SistemaContextEngineeringAvanzado()
# resultado = sistema_avanzado.consultar("¿Qué es Python?")
# sistema_avanzado.registrar_feedback("consulta_1", True, "Muy útil")
# print(sistema_avanzado.obtener_metricas())

## 🔥 Ejercicio de Desafío 3.3 (Opcional)

Crea un sistema de context engineering de nivel empresarial que incluya:

1. **Ingestión automática**: Cargar documentos desde múltiples fuentes (S3, Google Drive, APIs)
2. **Pipeline de procesamiento**: Limpieza, normalización, enriquecimiento automático
3. **Versionado de documentos**: Trackear cambios en documentos y re-indexar cuando sea necesario
4. **Sistema de permisos**: Control de acceso a diferentes documentos según usuario
5. **Dashboard de analytics**: Visualización de uso, popularidad de documentos, calidad de respuestas

**Desafío extra**: Implementa un sistema de aprendizaje que mejore automáticamente los prompts basándose en feedback histórico.

---

# 📝 Reflexión y Mejores Prácticas

## ✅ ¿Qué Aprendimos?

### 1. RAG Básico
- **Embeddings**: Representaciones vectoriales capturan significado semántico
- **Búsqueda semántica**: Encontrar información por significado, no solo palabras clave
- **Chunking**: Dividir documentos grandes preservando contexto
- **Contexto en prompts**: Añadir información relevante mejora respuestas

### 2. RAG Avanzado
- **Múltiples fuentes**: Combinar diferentes tipos de datos
- **Agregación inteligente**: Decidir qué fuentes usar según la pregunta
- **Filtrado**: Usar metadatos para refinar búsquedas
- **Re-ranking**: Mejorar resultados iniciales

### 3. Sistemas Completos
- **Caché**: Optimizar costos y latencia
- **Validación**: Verificar calidad de respuestas
- **Métricas**: Monitorear performance del sistema
- **Feedback loops**: Mejorar continuamente

## ⚠️ Errores Comunes a Evitar

1. **Chunks demasiado pequeños o grandes**
   - ❌ Chunks de 50 palabras (pierden contexto)
   - ✅ Chunks de 200-500 palabras con overlap

2. **Sin validación de respuestas**
   - ❌ Confiar ciegamente en lo que genera el modelo
   - ✅ Validar que la respuesta es relevante y precisa

3. **Ignorar metadatos**
   - ❌ Buscar en todos los documentos siempre
   - ✅ Usar filtros para reducir búsqueda

4. **Sin caché de embeddings**
   - ❌ Recalcular embeddings cada vez
   - ✅ Cachéar embeddings de documentos estáticos

5. **Prompts no optimizados**
   - ❌ Prompt genérico para todas las preguntas
   - ✅ Adaptar prompt según tipo de pregunta y contexto

## 🚀 Tips para Producción

1. **Chunking estratégico**: 
   - Documentos técnicos: 300-500 palabras
   - Conversaciones: Por turno completo
   - Código: Por función/clase

2. **Optimización de embeddings**:
   - Usar modelos más pequeños para documentos
   - Modelos más grandes solo para queries críticas
   - Batch processing para múltiples documentos

3. **Búsqueda híbrida**:
   - Combinar búsqueda semántica + keyword
   - Re-ranking con múltiples señales
   - Filtrado por metadatos antes de búsqueda

4. **Monitoreo**:
   - Trackear latencia de cada componente
   - Monitorear calidad de respuestas (feedback)
   - Alertas para degradación de performance

5. **Escalabilidad**:
   - Usar vector stores especializados (Pinecone, Weaviate)
   - Implementar sharding para grandes volúmenes
   - Caché distribuido para alta concurrencia

6. **Seguridad**:
   - Validar inputs de usuario
   - Sanitizar documentos antes de indexar
   - Control de acceso a documentos sensibles

## 📚 Recursos Adicionales

- [LangChain RAG Tutorial](https://python.langchain.com/docs/use_cases/question_answering/)
- [Pinecone Vector Database](https://www.pinecone.io/learn/)
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)
- [RAG Best Practices](https://www.pinecone.io/learn/retrieval-augmented-generation/)

---

## 🎓 Conclusión del Curso

¡Felicitaciones por completar los 3 módulos del curso! 🎉

Has aprendido:
- ✅ Fundamentos de prompt engineering
- ✅ Técnicas avanzadas para sistemas
- ✅ Context engineering y RAG

**Próximos pasos sugeridos**:
1. Implementa un proyecto real usando estas técnicas
2. Experimenta con diferentes modelos y configuraciones
3. Únete a comunidades de prompt engineering
4. Mantente actualizado con las últimas técnicas

¡Éxito en tus proyectos con IA! 🚀